In [52]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [53]:
data = pd.read_excel("SET50 1 Day.xlsx",
    sheet_name=0,
    header=0,
    index_col="Date",
    keep_default_na=True
)

In [54]:
data["y"] = data["Close"].shift(periods=-1)
data = data.drop("Volume",axis = 1)
data = data.dropna()
data_np = data.to_numpy()
data["Date"] = data.index
data.index = np.arange(-9, len(data) -9)

In [55]:
data

,Open,High,Low,Close,y,Date
-9,969.10,970.80,966.90,967.10,972.20,1996-06-17
-8,967.10,973.00,964.90,972.20,966.80,1996-06-18
-7,972.40,975.50,966.30,966.80,956.60,1996-06-19
-6,966.80,968.80,956.40,956.60,956.30,1996-06-20
-5,956.40,959.00,953.70,956.30,950.70,1996-06-21
...,...,...,...,...,...,...
6793,1048.78,1062.44,1046.73,1058.03,1056.32,2019-01-17
6794,1058.03,1063.75,1055.77,1056.32,1060.95,2019-01-18
6795,1056.32,1062.23,1056.04,1060.95,1066.79,2019-01-19
6796,1062.75,1069.80,1061.81,1066.79,1068.26,2019-01-21


In [56]:
def windowed_dataset(data_np, timestep = 10):
    X = np.array(data_np[0:timestep,:-1])
    X = np.expand_dims(X, axis=-1)
    X = np.reshape(X,(1,timestep,4))
    for i in range(data_np.shape[0] - timestep + 1):
        add = data_np[i:timestep+i,:-1]
        add = np.expand_dims(add, axis=-1)
        add = np.reshape(add,(1,timestep,4))
        X = np.concatenate((X,add),axis=0)
    Y = data_np[timestep:,-1]
    return X,Y

In [57]:
X,Y = windowed_dataset(data_np)

In [58]:
loaded_model = tf.keras.models.load_model('my_model')

In [59]:
def predict(index):
    return loaded_model.predict(X[index:index+1,:,:])[0,0]

In [60]:
# ใส่ index ของวันที่จะทำนาย เช่นวันที่ 2019-01-17 จะมี index ใน dataframe data คือ 6793
# จะได้ค่า Close ของวันถัดไป (2019-01-18) มาโดยใช้ข้อมูลของ 10 วันก่อนหน้า (2019-01-08 - 2019-01-17)

predict(6793)

1048.4877